In [15]:
import pandas as pd
import numpy as np
import os
import re
import csv
import requests

In [11]:
pc_file = os.path.join(os.pardir,"data","validated_postcodes_uniq.csv")
pcdf = pd.read_csv(pc_file, header=None, names={'postalcode'})
pcdf = pcdf.fillna('')
pcdf['postalcode']

0        400336
1        546183
2        578345
3        650420
4        460059
5        466507
6        357984
7        760776
8        560129
9        732685
10       669558
11       737785
12       330004
13       531470
14       684686
15       760150
16       640489
17       798595
18       828785
19       540142
20       543124
21       550407
22       670183
23       510709
24       760797
25       315079
26       275018
27       310153
28       160043
29       270026
          ...  
27097    454965
27098    549740
27099    428717
27100    535039
27101    746760
27102    577591
27103    460449
27104    548135
27105    554863
27106    455301
27107    598176
27108    489204
27109    381006
27110    117846
27111    788588
27112    508254
27113    259731
27114    759711
27115    249446
27116    808631
27117    574148
27118    398231
27119    786442
27120    596548
27121    538908
27122    218695
27123    229236
27124    487448
27125    578291
27126    637923
Name: postalcode, dtype:

In [12]:
pc_latlon_file = os.path.join(os.pardir,"data", "pc_latlon.csv")
pc_latlon_df = pd.read_csv(pc_latlon_file)
pc_latlon_df['postalcode']

Series([], Name: postalcode, dtype: object)

In [19]:
def onemap_geocode(postalcode):
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    onemap_request = 'https://developers.onemap.sg/commonapi/search?searchVal=' + str(postalcode) + '&returnGeom=Y&getAddrDetails=N'
#     print(onemap_request)
    response = requests.get(onemap_request)
    response_json = response.json()

    try:
        lat = response_json['results'][0]['LATITUDE']
        lon = response_json['results'][0]['LONGITUDE']
    except IndexError:
        lat = ''
        lon = ''
#     print('response for ' + str(postalcode) + ': ' + lat + ", " + lon)
    return lat + ', ' + lon

In [21]:
%%time
geocode_counter = 0

with open(pc_latlon_file, 'a', newline='') as outfile:
    writer = csv.writer(outfile)
    for postalcode in pcdf['postalcode']:
        if postalcode not in pc_latlon_df['postalcode']:

            latlon = onemap_geocode(postalcode)
            lat = latlon.split(',')[0]
            lon = latlon.split(',')[1]

            newrow = [postalcode, lat, lon]
            writer.writerow(newrow)
            geocode_counter += 1
            print('#' + str(geocode_counter) + ": " + str(newrow))

#1: [400336, '1.32572020348', ' 103.901822393']
#2: [546183, '1.3624250511', ' 103.876520751']
#3: [578345, '1.37403135668', ' 103.832480498']


KeyboardInterrupt: 